In [74]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as f
import matplotlib.pyplot as plt

In [75]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [76]:
num_epochs = 5
batch_size =4
learning_rate =0.001

In [77]:
# dataset has pil image of range [0,1]
# we tranform them to tensors of normalized range [-1,1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [78]:
train_dataset = torchvision.datasets.CIFAR10(root = './',train=True,download=True,transform=transform)

Files already downloaded and verified


In [79]:
test_dataset = torchvision.datasets.CIFAR10(root = './',train=False,download=True,transform=transform)

Files already downloaded and verified


In [80]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [81]:
classes = ('plane','car','bird','cat','deer','dog','horse','ship','truck','frog')

In [88]:
# implement conv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)# 3 for input channel size is 3, output channel size is 6 and kernel size is 5
        self.pool = nn.MaxPool3d(2,2)# kernel size of 2 and stride of 2
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)# fully connected layer
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x=self.pool(f.relu(self.conv1(x)))
        x=self.pool(f.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)
        x=f.relu(self.fc1(x))
        x=f.relu(self.fc2(x))
        x=self.fc3(x)
        return x
        

In [83]:
model = ConvNet().to(device)

In [84]:
criterion = nn.CrossEntropyLoss()
optimizer =torch.optim.SGD(model.parameters(),lr=learning_rate)

In [85]:
n_total_steps = len(train_loader)

In [90]:
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        # original shape: [4,3,32,32] = 4,3,1024
        # input layer: 3 input channels, 6 output chanels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        #backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000==0:
            print(f'epoch [{epoch+1}/{num_epochs}], step [{i+1}/{n_total_steps}], loss: {loss.item():.4f}')

print("finished training")

RuntimeError: Given groups=1, weight of size [16, 6, 5, 5], expected input[4, 3, 14, 14] to have 6 channels, but got 3 channels instead

In [87]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct=[0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        #max returns (value,index)
        _,predicted = torch.max(outputs,1)
        n_samples+=labels.size()
        n_correct+=(predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label]+=1
            n_class_samples[label]+=1
    
    acc = 100*n_correct/n_samples
    print(f'Accuracy of the network: {acc}%')

    for i in range(10):
        acc = 100.0 +n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of {classes[i]}:{acc}%')

RuntimeError: Given groups=1, weight of size [16, 6, 5, 5], expected input[4, 3, 14, 14] to have 6 channels, but got 3 channels instead